In [ ]:
!pip install torch-geometric

In [ ]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch_geometric import nn as gnn

from matplotlib import pyplot as plt

In [ ]:
splits = ["train", "valid", "test"]

In [ ]:
layout_nlp_default = '/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/default'
layout_nlp_random = '/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/random'
layout_xla_default = '/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/xla/default'
layout_xla_random = '/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/xla/random'

tile_xla = '/kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla'

In [ ]:
def load_n_data_to_df(directory, split, n, pos=0):
    
    path = os.path.join(directory, split)
    files = os.listdir(path)
    
    data_list = []
    
    n = min(n, len(files))
    for file in tqdm(files[pos:pos + n]):
        file_path = os.path.join(path, file)
        model_graph = dict(np.load(file_path))
        model_graph["file"] = file
        data_list.append(model_graph)
    
    return pd.DataFrame(data_list)

In [ ]:
def load_data_to_df(directory, split):
    
    n = len(os.listdir(os.path.join(directory, split)))
    return load_n_data_to_df(directory, split, n)

In [ ]:
df_layout = load_n_data_to_df(layout_nlp_random, "valid", 5)

In [ ]:
for i in range(6):
    print(len(df_layout.iloc[3, i]))

print(len(df_layout.iloc[3, 3][0]))
df_layout.iloc[3]

In [ ]:
del df_layout

In [ ]:
df_train = load_data_to_df(tile_xla, "train")
df_valid = load_data_to_df(tile_xla, "valid")
df_test = load_data_to_df(tile_xla, "test")

In [ ]:
class GAT(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super().__init__()
        self.conv1 = gnn.GATConv(in_channels, hidden_channels, heads, dropout=0.6)
        # On the Pubmed dataset, use `heads` output heads in `conv2`.
        self.conv2 = gnn.GATConv(hidden_channels * heads, out_channels, heads=1,
                             concat=False, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x


In [ ]:
class TileModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        op_embedding_dim = 8
        self.opcode_embedding = nn.Embedding(120, op_embedding_dim)
        
        node_feature_dim = 140
        
        conv_out_dim = 48
        
        
        # hidden_channels = 8
        # heads = 8
        # self.conv = GAT(op_embedding_dim + node_feature_dim, hidden_channels, conv_out_dim, heads)
        
        # self.conv = gnn.GCNConv(op_embedding_dim + node_feature_dim, conv_out_dim)
        
        hidden_channels = 120
        num_layers = 4
        self.conv = gnn.GraphSAGE(op_embedding_dim + node_feature_dim, hidden_channels, num_layers, conv_out_dim)
        
        config_dim = 24
        self.fwd = nn.Sequential(
            nn.Linear(conv_out_dim + config_dim, 48),
            nn.ReLU(),
            nn.Linear(48, 48),
            nn.ReLU(),
            nn.Linear(48, 1)
        )
        
        
    def forward(self, node_feat, node_opcode, edge_index, config_feat):
        """
            Shapes:
                node_feat    - (n, 140)
                node_opcode  - (n, )
                edge_index   - (m, 2)
                config_feat  - (c, 24)
            
            Approach:
                1. Opcode embeddings
                2. Concatenate embeddings to node feature-vector
                3. Convolutional layer for node embeddings
                4. Pooling for graph embedding
                5. Concatenate configuration feature-vector to graph embedding
                6. Forward layer
                7. Flatten
            
            Approach is inline with the paper Phitchaya Mangpo Phothilimthana et. al (2023) 
        """
        node_opcode_embedding = self.opcode_embedding(node_opcode) # (n, 8)
        
        x = torch.concat([node_feat, node_opcode_embedding], dim = 1) # (n, 148)
        
        x = self.conv(x, edge_index) # (n, 48)
                
        x = torch.mean(x, 0) # (48, )    
            
        x = x.repeat(len(config_feat), 1) # (c, 48)
        
        x = torch.concat([x, config_feat], dim = 1) # (c, 72)
        
        x = self.fwd(x) # (c, 1)
        
        return torch.flatten(x) # (c, )
        

In [ ]:
class TileDataset(Dataset):
    
    def __init__(self, tiles):
        self.tiles = tiles
    
    def __len__(self):
        return len(self.tiles)
    
    def __getitem__(self, idx):
        
        file = self.tiles.iloc[idx]['file']
        node_feat = torch.from_numpy(self.tiles.iloc[idx]['node_feat'])
        node_opcode = torch.from_numpy(self.tiles.iloc[idx]['node_opcode']).type(torch.int64)
        edge_index = torch.from_numpy(self.tiles.iloc[idx]['edge_index']).permute(1, 0)
        config_feat = torch.from_numpy(self.tiles.iloc[idx]['config_feat'])
        config_runtime = torch.from_numpy(self.tiles.iloc[idx]['config_runtime'])
        config_runtime_normalizers = torch.from_numpy(self.tiles.iloc[idx]['config_runtime_normalizers'])
        
        return {
            "file": file,
            "node_feat": node_feat,
            "node_opcode": node_opcode,
            "edge_index": edge_index,
            "config_feat": config_feat,
            "y": config_runtime / config_runtime_normalizers
        }
    
    def __iter__(self):
        self.i = 0
        return self
    
    def __next__(self):
        if self.i < len(self.tiles):
            item = self[self.i]
            self.i += 1
            return item
        else:
            raise StopIteration
    
train_dataset = TileDataset(df_train)
valid_dataset = TileDataset(df_valid)
test_dataset = TileDataset(df_test)

In [ ]:
model = TileModel()

data = train_dataset[0]

model(data['node_feat'], data['node_opcode'], data['edge_index'], data['config_feat'])

In [ ]:
# Count the number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters in the model: {total_params}")

## Training

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = TileModel().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 10

train_losses = np.empty((0,))
valid_losses = np.empty((0,))
    


def train():
    model.train()
    
    epoch_train_losses = np.empty((0,))
    for tile in train_dataset:
        
        node_feat, node_opcode, edge_index, config_feat = tile["node_feat"].to(device), tile["node_opcode"].to(device), tile["edge_index"].to(device), tile["config_feat"].to(device)
        
        optimizer.zero_grad()
        pred = model(node_feat, node_opcode, edge_index, config_feat).to("cpu")
        loss = loss_fn(pred, tile["y"])
                
        loss.backward()
        optimizer.step()
        
        epoch_train_losses = np.append(epoch_train_losses, loss.item())
    return np.mean(epoch_train_losses)

def test():
    model.eval()
    
    epoch_valid_losses = np.empty((0,))
    
    with torch.no_grad():
        for tile in valid_dataset:
                
            node_feat, node_opcode, edge_index, config_feat = tile["node_feat"].to(device), tile["node_opcode"].to(device), tile["edge_index"].to(device), tile["config_feat"].to(device)
    
            pred = model(node_feat, node_opcode, edge_index, config_feat).to("cpu")
            loss = loss_fn(pred, tile["y"])
            
            epoch_valid_losses = np.append(epoch_valid_losses, loss.item())
    return np.mean(epoch_valid_losses)


for i in range(1, epochs + 1):
    
    train_loss = train()
    valid_loss = test()
    
    train_losses = np.append(train_losses, train_loss)
    valid_losses = np.append(valid_losses, valid_loss) 
    print(f"Epoch: {i}, Train Loss: {train_losses[-1]}, Valid Loss: {valid_losses[-1]}")


In [ ]:
# Store model
torch.save(model.state_dict(), "tile_model.pt")

In [ ]:
# Plot losses
all_epochs = list(range(epochs))

# create figure and axis objects with subplots()
fig,ax = plt.subplots()
# make a plot
ax.plot(all_epochs,
        valid_losses,
        color="red", 
        marker="o")
# set x-axis label
ax.set_xlabel("Epoch", fontsize = 14)
# set y-axis label
ax.set_ylabel("Valid",
              color="red",
              fontsize=14)

# twin object for two different y-axis on the sample plot
ax2=ax.twinx()
# make a plot with different y-axis using second axis object
ax2.plot(all_epochs, train_losses,color="blue",marker="o")
ax2.set_ylabel("Train",color="blue",fontsize=14)
plt.show()


In [ ]:
def predict(tile, model):
    model = model.to(device)
    node_feat = tile["node_feat"].to(device)
    node_opcode = tile["node_opcode"].to(device)
    edge_index = tile["edge_index"].to(device)
    config_feat = tile["config_feat"].to(device)
    
    out = model(node_feat, node_opcode, edge_index, config_feat).to("cpu")
    return torch.sort(out).indices[:5]
    

In [ ]:
def get_predictions_per_file(model, dataset):
    
    if isinstance(model, str):
        state_dict = torch.load(model)
        model = TileModel()
        model.load_state_dict(state_dict)
    
    model.eval()
    prediction_for_file = {}
    
    with torch.no_grad():
        for tile in dataset:
            prediction_for_file[tile["file"]] = predict(tile, model)

    return prediction_for_file

In [ ]:
def evaluate_model(model):
    
    # Make predictions
    predictions = list(get_predictions_per_file(model, valid_dataset).values())

    # Calculate score
    scores = np.empty((0,))

    for i, tile in tqdm(enumerate(valid_dataset), total=len(valid_dataset)):
        best_prediction = min([valid_dataset[i]["y"][pred_ind] for pred_ind in predictions[i][:5]])
        best_total = min(valid_dataset[i]["y"])
        scores = np.append(scores, 2.0 - best_prediction / best_total)

    avg_score = np.mean(scores)
    print("Score:", avg_score)
    return avg_score
    

In [ ]:
evaluate_model(model)

In [ ]:
def create_submission():
    
    submission = pd.read_csv("/kaggle/input/predict-ai-model-runtime/sample_submission.csv")
    
    predictions = get_predictions_per_file("tile_model.pt", test_dataset)
    
    for model_name in predictions.keys():
        model_id = 'tile:xla:' + model_name[:-4]
        submission.loc[submission["ID"] == model_id, "TopConfigs"] = ";".join([str(pred) for pred in predictions[model_name].tolist()])
    
    submission.to_csv("submission.csv", index=False)
        
create_submission()